<참조링크>
* http://pythonstudy.xyz/python/article/307-Django-%ED%85%9C%ED%94%8C%EB%A6%BF-Template
* https://docs.djangoproject.com/
* https://wikidocs.net/70736


### 목차
1. 개발환경 준비
    * 가상환경(python)
    * Django 설치
    * 프로젝트 생성
    * 실행
2. Django 훑어보기
    * APP
    * URL
    * View
    * 모델
    * 
    
---

1. 가상환경 설치 및 프로젝트 생성하기
    * 가상환경 설치
        ```
            -- 설치할 디렉토리 생성
            C:\Users\pahkey> cd \
            C:\> mkdir venvs
            C:\> cd venvs

            -- 파이썬 가상환경 등록
            C:\venvs> python -m venv mysite

            -- 가상환경 로드 및 종료
                설치경로/Script activate OR deacitvate
        ```

    * Django 설치
        ```
            pip install django==3.1.3
            c:\workspace\venvs\mysite\scripts\python.exe -m pip install --upgrade pip
            (pip 업그레이드 필요시)
        ````
        
    * 프로젝트 생성
        ```
            django-admin startproject config '적용할 경로'
        ```
        
    * 실행
        ```
            python manage.py runserver
            # Starting development server at http://127.0.0.1:8000/
            # cntl + 주소 클릭
        ```

### Django 흝어보기
* 모델
    * 내용: 파이썬 객체로서 데이터베이스 생성/저장/연동 기능 포함
    * 설계: 
      * "from django.db import models"를 이용하여 원하는 기능을 하는 class 생성
      * 모델을 생성하면 데이터베이스를 자동으로 생성해주는 유틸리티 실행

          `python manage.py makemigrations` \
          makemigrations : 생성 가능한 모델을 찾아 테이블이 존재하지 않을 경우 마이그레이션을 생성

          `python manage.py migrate`  \
          migrate: 마이그레이션을 실행하고 사용자의 데이터베이스에 테이블을 생성

    * 사용 예:
    ```python
        # Import the models we created from our "news" app
        >>> from news.models import Article, Reporter
    
        # No reporters are in the system yet.
        >>> Reporter.objects.all()
        <QuerySet []>
    
        # Create a new Reporter.
        >>> r = Reporter(full_name="John Smith")
    
        # Save the object into the database. You have to call save() explicitly.
        >>> r.save()
    
        # Now it has an ID.
        >>> r.id
        1
    ```
---

* APP
    * 내용: 한 프로젝트를 구성하는 하위 단위모듈
    * 구성: 자기 자산만의 모델, 뷰, 탬플릿, URL 매핑등을 독자적으로 가지고 있음
    * 생성: 
        `./manage.py startapp home`
---

* URL 설계(urls.py)
    * 내용: 개발자에 의해 이미 만들어진 URL 패턴들의 정보 위치
            사용자가 특정 URL을 요청하면 일치하는 패턴의 URL 정보를 콜백함수(view)에 전달
        * path( "URL 경로"(route), 콜백함수(views), 키워드(kwrgs), 이름지정(name) )
    * 사용 예:
    
        ```python
        from django.urls import path
        from . import views

            urlpatterns = [
            path("articles/<int:year>/", views.year_archive),
            path("articles/<int:year>/<int:month>/", views.month_archive),
            path("articles/<int:year>/<int:month>/<int:pk>/", views.article_detail),
            ]
        ```
---

* 뷰(view.py)
    * 내용: 요청된 URL 파라메터를 작동함수에 적용하여 tamplate를 rendering
    * view.year_archive 예
    
        ```python
        from django.shortcuts import render
        from .models import Article
        
        def year_archive(request, year):
            a_list = Article.objects.filter(pub_date__year=year)
            context = {"year": year, "article_list": a_list}
            return render(request, "news/year_archive.html", context)
        ```
        * request
        * render(요청된객체,템플릿,선택적인수)

---

* 템플릿(template)
    * 사전적 의미: 어떤 특정한 모양을 만들기 위한 틀
    * 내용: 이미 구성된 웹페이지(html) 파일로서 view로부터 전달된 데이터를 적용하여 표출
    * View로부터 어떤 데이타를 전달받아 HTML 템플릿 안에 그 데이타를 동적으로 치환해서 사용 --> 템플릿 언어가 따로 있음\
        * 변수: {{변수명, 변수명.속성}}
        * 태그: {{% 템플릿 태그 %}} 
            ```
                {% if count > 0 %}
                    Data Count = {{ count }}
                {% else %}
                    No Data
                {% endif %}

                {% for item in dataList %}
                  <li>{{ item.name }}</li>
                {% endfor %}

                {% csrf_token %}
            ``` 
        * 필터: 
            ```
                날짜 포맷 지정
                {{ createDate|date:"Y-m-d" }}

                소문자로 변경
                {{ lastName|lower }}
            ```
        * 코멘트 
            * 한줄: {# 내용 #}
            * 여러줄: {% comment %}, {% endcomment %}
        * HTML escape: HTML 내용 중에 <, >, ', ", & 등과 같은 문자들이 있을 경우
            * autoescape 태그
                ```
                    {% autoescape on %}
                        {{ content }}
                    {% endautoescape %}
                ```
            * escape 필터
                ```
                    {{ content|escape }}
                ```

    * 템플릿 셋팅
        ```
            settings.py의 TEMPLATES 섹션
            BACKEND: django.template.backends.django.DjangoTemplate(기본)
            DIRS: [BASE_DIR / 'templates']
        ```
    * 보통 `(예: /home/templates/)` 와 같이 템플릿 모음 경로를 설정하고 각 앱에서 호출(중복 방지)